#Exercise 2. Diffusion in RxD. An example of bistable dynamics allowing wave propagation
$$\frac{\partial u}{\partial t} = -ur(1-u)(0.3-u) + \sigma\frac{\partial^2u}{\partial x^2}$$

In [1]:
from neuron import h, rxd
import time
from pylab import *
from ipywidgets import widgets, fixed, Layout

In [2]:
%%html
<style type='text/css'>
.widget-inline-hbox .widget-label {
      max-width: 250px;
      min-width: 250px;
}
</style>

In [3]:
#This is the initialization of the RxD system - should be run only once
h.load_file('stdrun.hoc')                                     #A default NEURON library

dend = h.Section()                                            #Create a compartment called dend
dend.nseg = 101                                               #Divide the compartment to 101 segments

r = rxd.Region(h.allsec())                                    #Region r where the dynamics will take place

r_param = rxd.Parameter(r, initial=0)                         #An RxD parameter affecting the rate of u flux
u = rxd.Species(r, d=1, initial=0)                            #An arbitrary species called u, with diffusion 1 mM/(ms um^2)
# The "reaction" that takes place is a positive (when 0.3 < u < 1.0) or negative (otherwise)
# flux of species u (the magnitude of the flux is given as -u*(1-u)*(0.3-u))
# Other than that, only diffusion across the cylinder axis takes place
bistable_reaction = rxd.Rate(u, -u * r_param * (1 - u) * (0.3 - u))

vec_t = h.Vector()                                            #Create vectors for recordings
vec_u = h.Vector()
vec_t.record(h._ref_t)
vec_u.record(u.nodes(dend)(0.5)[0]._ref_concentration)        #Record the concentration variables

1.0

In [4]:
#This is the main simulation function of the RxD system. 
def runneuron(rate_param, fraction_initial):
    my_start_time = time.time()
    r_param.initial = rate_param
    h.finitialize()                                           #Initialize the dynamic variables
    for node in u.nodes:                                      #Set the concentration to 1
        if node.x < fraction_initial: node.concentration = 1  #in the proximal (first 20) segments
    
    f,ax = plt.subplots(1,1)
    data_u = array(vec_u)

    ax.plot(dend.L*array(u.nodes.x), u.nodes.concentration, 'r')
    cols = ['r','m','b','c','g']
    for i in range(1, 5):
        h.continuerun(i * 25)                                 #Run for (an additional) 25 milliseconds
        ax.plot(dend.L*array(u.nodes.x), u.nodes.concentration, cols[i], label = str(i*25)+' ms')
    
    ax.set_xlabel('$x$ ($\mu$m)')
    ax.set_ylabel('$c$ (mmol/l)')
    ax.legend()
    plt.suptitle("Simulation run in "+str(time.time() - my_start_time)+" seconds.")
    plt.show()

In [5]:
slider = widgets.interact(runneuron, rate_param=widgets.FloatSlider(min=0.0,max=2.0,step=0.01,value=0.0,description='Rate parameter $r$',layout=Layout(width='50%'), continuous_update = False),
                                     fraction_initial=widgets.FloatSlider(min=0.01,max=1.0,step=0.01,value=0.2,description='Fraction of initially activated',layout=Layout(width='50%'), continuous_update = False))

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Rate parameter $r$', layout…

#Question: Make the system propagate a wave by changing the rate parameter r. Change the diffusion parameter 10-fold larger (the Jupyter kernel has to be restarted after this change). How is the wave propagation affected?